# PROJEKT MODELU REGRESYJNEGO PRZEWIDYWANIA CEN MIESZKAŃ NA PODSTAWIE #

# ALGORYTMU LGBM #

  ## KWIECIEŃ 2025 ##

In [1]:
import pandas as pd
from pycaret.regression import setup, pull, compare_models, plot_model, load_model
import pymysql
from sqlalchemy import create_engine
import numpy as np
from scipy.stats import skewnorm
import matplotlib.pyplot as plt
import seaborn as sns
from joblib import parallel_backend

In [2]:
# tą komórkę uruchom jeżeli bazie bierzesz np. z DBeaver
# username = 'root'
# password = '1234'
# host = '127.0.0.1'
# port = 3306  
# database = 'projekt1'
# engine = create_engine(f'mysql+pymysql://{username}:{password}@{host}:{port}/{database}')

# df = pd.read_sql("SELECT * FROM saleflats", con=engine)

# engine.dispose()

In [3]:
# tę komórkę uruchom jeśli czerpiesz dane z pliku .csv
df = pd.read_csv('uzupelnione_mieszkania.csv', sep=',')

In [4]:
df

,SaleId,OriginalId,PortalId,Title,Description,Area,Price,OfferPrice,RealPriceAfterRenovation,OriginalPrice,...,Archive,Location,VoivodeshipNumber,CountyNumber,CommunityNumber,KindNumber,RegionNumber,SubRegionNumber,StreetNumber,EncryptedId
0,88,NaN,14,Mieszkanie trzypokojowe na sprzedaż,Mieszkanie o powierzchni 73m2 znajduje się na ...,73.00,766500.0,NaN,NaN,NaN,...,NaN,"Podlaskie, Białystok, Bojary",20.0,61.0,1.0,1.0,922490.0,NaN,NaN,02TgImo1wm-Ra4zUQxoPBw==
1,99,NaN,19,Sprzedam mieszkanie na parterze 64.8m2 Białyst...,Sprzedam mieszkanie na parterze 64.8m2 w 3-pię...,64.80,540000.0,NaN,NaN,NaN,...,NaN,"Podlaskie, Białystok, Antoniuk",20.0,61.0,1.0,1.0,922426.0,NaN,NaN,D-NUdVbtN3_kIW7ExgXkvA==
2,115,NaN,3,"Mieszkanie bezczynszowe, 3 pokoje, 2 łazienki",SPRZEDAŻ WYŁĄCZNIE BEZPOŚREDNIA. Agencjom nier...,51.00,540000.0,NaN,NaN,NaN,...,NaN,"Podlaskie, Białystok, Bagnówka",20.0,61.0,1.0,1.0,922461.0,NaN,NaN,lmBSU1XiXLSLOq1yKSe2kA==
3,140,NaN,14,Mieszkanie trzypokojowe na sprzedaż,***Oferta bez prowizji biura i podatku PCC!***...,67.62,544000.0,NaN,NaN,NaN,...,NaN,"Podlaskie, Białystok, Ul. Józefa Karola Puchal...",20.0,61.0,1.0,1.0,922410.0,NaN,43141.0,HWachf5gb6Vq-sFYH4YWnA==
4,145,NaN,3,3 Pokoje- 48M2-Osiedle Dziesięciny,Przedstawiamy na sprzedaż 3 pokojowe mieszkani...,48.00,459000.0,NaN,NaN,NaN,...,NaN,"Podlaskie, Białystok, Ul. Dziesięciny",20.0,61.0,1.0,1.0,922410.0,NaN,4656.0,AWsmt4ZY0X7lNv-6fSCn9Q==
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1133882,4418401,NaN,2,Mieszkanie na sprzedaż - 59 m² - 3 pokoje Grod...,Poznaj naszą inwestycję NA WSPÓLNEJ zlokalizow...,59.99,NaN,NaN,NaN,NaN,...,NaN,"Mazowieckie, Grodziski, Grodzisk Mazowiecki, W...",14.0,5.0,4.0,5.0,2594.0,NaN,24885.0,Ogpi4Jwj8KWDyT0rM_vFYftiMRU9ErLfA6XOKdBw9jY=
1133883,4418402,NaN,2,Mieszkanie na sprzedaż - 41 m² - 2 pokoje Grod...,Poznaj naszą inwestycję NA WSPÓLNEJ zlokalizow...,41.23,NaN,NaN,NaN,NaN,...,NaN,"Mazowieckie, Grodziski, Grodzisk Mazowiecki, W...",14.0,5.0,4.0,5.0,2594.0,NaN,24885.0,Ogpi4Jwj8KWDyT0rM_vFYa7D8z7yjun2jGP_aZIReN8=
1133884,4418404,NaN,2,Mieszkanie na sprzedaż - 49 m² - 3 pokoje Grod...,Poznaj naszą inwestycję NA WSPÓLNEJ zlokalizow...,49.09,NaN,NaN,NaN,NaN,...,NaN,"Mazowieckie, Grodziski, Grodzisk Mazowiecki, W...",14.0,5.0,4.0,5.0,2594.0,NaN,24885.0,Ogpi4Jwj8KWDyT0rM_vFYf9eM73_U8S05NBjfLiez6E=
1133885,4418405,NaN,2,Mieszkanie na sprzedaż - 49 m² - 3 pokoje Grod...,Poznaj naszą inwestycję NA WSPÓLNEJ zlokalizow...,49.48,NaN,NaN,NaN,NaN,...,NaN,"Mazowieckie, Grodziski, Grodzisk Mazowiecki, W...",14.0,5.0,4.0,5.0,2594.0,NaN,24885.0,Ogpi4Jwj8KWDyT0rM_vFYRXVZdKE7iJxxFPu0Bq0ikc=


In [5]:
df.head(10)

,SaleId,OriginalId,PortalId,Title,Description,Area,Price,OfferPrice,RealPriceAfterRenovation,OriginalPrice,...,Archive,Location,VoivodeshipNumber,CountyNumber,CommunityNumber,KindNumber,RegionNumber,SubRegionNumber,StreetNumber,EncryptedId
0,88,NaN,14,Mieszkanie trzypokojowe na sprzedaż,Mieszkanie o powierzchni 73m2 znajduje się na ...,73.00,766500.0,NaN,NaN,NaN,...,NaN,"Podlaskie, Białystok, Bojary",20.0,61.0,1.0,1.0,922490.0,NaN,NaN,02TgImo1wm-Ra4zUQxoPBw==
1,99,NaN,19,Sprzedam mieszkanie na parterze 64.8m2 Białyst...,Sprzedam mieszkanie na parterze 64.8m2 w 3-pię...,64.80,540000.0,NaN,NaN,NaN,...,NaN,"Podlaskie, Białystok, Antoniuk",20.0,61.0,1.0,1.0,922426.0,NaN,NaN,D-NUdVbtN3_kIW7ExgXkvA==
2,115,NaN,3,"Mieszkanie bezczynszowe, 3 pokoje, 2 łazienki",SPRZEDAŻ WYŁĄCZNIE BEZPOŚREDNIA. Agencjom nier...,51.00,540000.0,NaN,NaN,NaN,...,NaN,"Podlaskie, Białystok, Bagnówka",20.0,61.0,1.0,1.0,922461.0,NaN,NaN,lmBSU1XiXLSLOq1yKSe2kA==
3,140,NaN,14,Mieszkanie trzypokojowe na sprzedaż,***Oferta bez prowizji biura i podatku PCC!***...,67.62,544000.0,NaN,NaN,NaN,...,NaN,"Podlaskie, Białystok, Ul. Józefa Karola Puchal...",20.0,61.0,1.0,1.0,922410.0,NaN,43141.0,HWachf5gb6Vq-sFYH4YWnA==
4,145,NaN,3,3 Pokoje- 48M2-Osiedle Dziesięciny,Przedstawiamy na sprzedaż 3 pokojowe mieszkani...,48.00,459000.0,NaN,NaN,NaN,...,NaN,"Podlaskie, Białystok, Ul. Dziesięciny",20.0,61.0,1.0,1.0,922410.0,NaN,4656.0,AWsmt4ZY0X7lNv-6fSCn9Q==
5,159,NaN,3,"Mieszkanie, 87 m², Białystok","Przestronne, jasne mieszkanie na zamkniętym os...",87.00,779000.0,NaN,NaN,NaN,...,NaN,"Podlaskie, Białystok, Ul. Adama Mickiewicza",20.0,61.0,1.0,1.0,922410.0,NaN,12740.0,SEPz6jrRB1vuWWYOn99h7g==
6,165,NaN,3,Przytulne mieszkanie blisko centrum!,Na sprzedaż 2 pokojowe mieszkanie położone na ...,34.10,359000.0,NaN,NaN,NaN,...,NaN,"Podlaskie, Białystok, Bojary",20.0,61.0,1.0,1.0,922490.0,NaN,16041.0,Gt5-LUpJNg-wBgBd8-hxGQ==
7,173,NaN,3,"**Jasne, 2 Pokojowe Mieszkanie, Widok Na Miasto**","Mieszkanie o powierzchni 35,7 m2 Ul. Jarzębino...",35.70,380000.0,NaN,NaN,NaN,...,NaN,"Podlaskie, Białystok",20.0,61.0,1.0,1.0,922410.0,NaN,7290.0,a5iP5JQyibefTTLgBRU8oA==
8,179,NaN,3,*Bezpośrednio*PIĘKNY APARTAMENT w Centrum Lipo...,***REZERWACJA ***WYJĄTKOWA PROPOZYCJA DLA WYMA...,39.40,559000.0,NaN,NaN,NaN,...,NaN,"Podlaskie, Białystok, Ul. Adama Mickiewicza",20.0,61.0,1.0,1.0,922410.0,NaN,12740.0,C_F7Bxl4hKQ2a2votBqs-g==
9,189,NaN,4,2-pokojowe mieszkanie na sprzedaż,Na sprzedaż 2 pokojowe mieszkanie położone na ...,34.20,354000.0,NaN,NaN,NaN,...,NaN,"Podlaskie, Białystok, Ul. Piasta",20.0,61.0,1.0,1.0,922410.0,NaN,16041.0,XylelQthYlIn_7k8orxBfQ==


In [6]:
df.sample(10)

,SaleId,OriginalId,PortalId,Title,Description,Area,Price,OfferPrice,RealPriceAfterRenovation,OriginalPrice,...,Archive,Location,VoivodeshipNumber,CountyNumber,CommunityNumber,KindNumber,RegionNumber,SubRegionNumber,StreetNumber,EncryptedId
264992,1916413,NaN,3,Osiedle Dzika Róża | mieszkanie 3-pok. | 51,3-pokojowe mieszkanie numer 51 na 6. piętrze w...,60.38,NaN,NaN,NaN,NaN,...,NaN,"Łódzkie, Kutnowski, Kutno",10.0,2.0,1.0,1.0,969103.0,NaN,NaN,SsJUrcwFsEQemZz60jV9PgGYSGvKlYoNawdyhAmLdNQ=
498062,2881646,NaN,3,Wygodne Mieszkanie Rezydencja Ogród Saski | A1.1,Oferta Przedmiotem ogłoszenia jest 1-pokojowe ...,30.98,NaN,NaN,NaN,NaN,...,NaN,"Lubelskie, Lublin, Ul. Wieniawska",6.0,63.0,1.0,1.0,954700.0,NaN,24121.0,zWvN8ezJKq1nPIN7JenEa_ID1O_Boi4SbTP5CDBseOg=
5519,36180,NaN,17,"Nowe mieszkanie Piotrowice-Ochojec, ul. Rzepak...","Lokalizacja: ul. Rzepakowa, Piotrowice-Ochojec...",85.97,799521.0,NaN,NaN,NaN,...,NaN,"Śląskie, Katowice",24.0,69.0,1.0,1.0,937474.0,NaN,19339.0,YzUkZIb3Jfer6btnlCaV0b1cwColzIKnjrh_K5zWcDs=
1076518,4316606,NaN,19,"Mieszkanie w Dębnie, Dębno koło Kostrzyn nad O...",Cena za dwa lokale Gotowe do zamieszkania od z...,40.00,289000.0,NaN,NaN,NaN,...,NaN,"Zachodniopomorskie, Myśliborski, Dębno",32.0,10.0,3.0,4.0,NaN,NaN,NaN,UZQNPa8wXDtjHQJZc2ViMVxslNBbpBprsR-39HjnMC8=
452859,2795798,NaN,11,"Mieszkanie, Wrocław, Śródmieście, 48 m²","Mieszkanie, Wrocław, Śródmieście, 48 m² Ogłosz...",48.00,655000.0,NaN,NaN,NaN,...,NaN,"Dolnośląskie, Wrocław, Wrocław-śródmieście, Śr...",2.0,64.0,6.0,9.0,1005034.0,NaN,NaN,F884Hs6ya0yQxGYLOSLZHP4qMueMoVmzI-akCGx4atA=
406581,2699353,NaN,3,2 Pokoje/ Rozkładowe/ Balkon/ Zamkniete Osiedle,Oferta dostępna wyłącznie w MK PRESTIGE NIERUC...,48.00,699000.0,NaN,NaN,NaN,...,NaN,"Dolnośląskie, Wrocław, Wrocław-krzyki, Ul. Ks....",2.0,64.0,3.0,9.0,986544.0,NaN,8540.0,T-4hYbFdNE7MdHlhK6O1nmM_a3KKvllm6VBpOMLzO-8=
644481,3211384,NaN,19,Mieszkanie Wieliczka os. Secesja,Mieszkanie poniżej 1km od granicy Krakowa. Odś...,53.00,662500.0,NaN,NaN,NaN,...,NaN,"Małopolskie, Kraków",12.0,61.0,NaN,NaN,NaN,NaN,NaN,xvm2RSqaBBQ7eXWbLv0cgo4lcJSA8OB9lFvDEJq2Tpw=
238451,1744098,NaN,2,"Narożne, dwustronne mieszkanie",!Bezpośrednio !Przestronne mieszkanie przy ul....,56.00,750000.0,NaN,NaN,780000.0,...,NaN,"Mazowieckie, Warszawa, Włochy, Ul. Komitetu Ob...",14.0,65.0,17.0,8.0,988862.0,NaN,52592.0,qrS8Bt6AaJbEMh8FfmdjIP4qMueMoVmzI-akCGx4atA=
932758,3980340,NaN,17,"Mieszkanie, ul. Niebyła",Mieszkanie na sprzedaż Kraków Wzgórza Krzesław...,63.00,829000.0,NaN,NaN,NaN,...,NaN,"Małopolskie, Kraków, Kraków-nowa Huta, Ul. Nie...",12.0,61.0,3.0,9.0,950718.0,NaN,14115.0,KMgdFqEu8QkP6Xt2w-dV-O7YjVk_597D3gSimczV9ho=
626471,3164929,NaN,11,"Mieszkanie, Bielsko-Biała, Kamienica, 70 m²","Mieszkanie, Bielsko- Biała, Kamienica, 70 m² O...",70.31,843720.0,NaN,NaN,NaN,...,NaN,"Śląskie, Bielsko-biała, Kamienica",24.0,61.0,1.0,1.0,923704.0,NaN,NaN,gfm75xpY-pxd3DYWmXaG-ObQxVp1jDeL4EC2z-A_Xv4=


In [7]:
df.info

<bound method DataFrame.info of           SaleId  OriginalId  PortalId  \
0             88         NaN        14   
1             99         NaN        19   
2            115         NaN         3   
3            140         NaN        14   
4            145         NaN         3   
...          ...         ...       ...   
1133882  4418401         NaN         2   
1133883  4418402         NaN         2   
1133884  4418404         NaN         2   
1133885  4418405         NaN         2   
1133886  4418406         NaN         2   

                                                     Title  \
0                      Mieszkanie trzypokojowe na sprzedaż   
1        Sprzedam mieszkanie na parterze 64.8m2 Białyst...   
2            Mieszkanie bezczynszowe, 3 pokoje, 2 łazienki   
3                      Mieszkanie trzypokojowe na sprzedaż   
4                       3 Pokoje- 48M2-Osiedle Dziesięciny   
...                                                    ...   
1133882  Mieszkanie na sprzed

In [8]:
df[df.duplicated()]

,SaleId,OriginalId,PortalId,Title,Description,Area,Price,OfferPrice,RealPriceAfterRenovation,OriginalPrice,...,Archive,Location,VoivodeshipNumber,CountyNumber,CommunityNumber,KindNumber,RegionNumber,SubRegionNumber,StreetNumber,EncryptedId


In [9]:
df.nunique()

SaleId                       1133887
OriginalId                         0
PortalId                          35
Title                         635825
Description                   982079
Area                           14360
Price                         131353
OfferPrice                     23232
RealPriceAfterRenovation       29481
OriginalPrice                  17584
PricePerSquareMeter           306308
NumberOfRooms                     90
BuiltYear                        281
Type                               2
BuildingType                       4
BuildingCondition                  4
OfferFrom                          2
Floor                             62
Floors                            77
TypeOfMarket                       2
OwnerType                         96
DateAddedToDatabase          1082247
DateAdded                    1066927
DateLastModification          242626
DateLastRaises                292764
NewestDate                   1070077
AvailableFrom                      0
L

In [10]:
correlation_matrix = df[['Area', 'Price', 'BuiltYear', 'Floor', 'Floors', 'CommunityScore']].corr()

In [11]:
plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Matrix of Price')

Text(0.5, 1.0, 'Correlation Matrix of Price')

In [12]:
df.isnull()

,SaleId,OriginalId,PortalId,Title,Description,Area,Price,OfferPrice,RealPriceAfterRenovation,OriginalPrice,...,Archive,Location,VoivodeshipNumber,CountyNumber,CommunityNumber,KindNumber,RegionNumber,SubRegionNumber,StreetNumber,EncryptedId
0,False,True,False,False,False,False,False,True,True,True,...,True,False,False,False,False,False,False,True,True,False
1,False,True,False,False,False,False,False,True,True,True,...,True,False,False,False,False,False,False,True,True,False
2,False,True,False,False,False,False,False,True,True,True,...,True,False,False,False,False,False,False,True,True,False
3,False,True,False,False,False,False,False,True,True,True,...,True,False,False,False,False,False,False,True,False,False
4,False,True,False,False,False,False,False,True,True,True,...,True,False,False,False,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1133882,False,True,False,False,False,False,True,True,True,True,...,True,False,False,False,False,False,False,True,False,False
1133883,False,True,False,False,False,False,True,True,True,True,...,True,False,False,False,False,False,False,True,False,False
1133884,False,True,False,False,False,False,True,True,True,True,...,True,False,False,False,False,False,False,True,False,False
1133885,False,True,False,False,False,False,True,True,True,True,...,True,False,False,False,False,False,False,True,False,False


In [13]:
df.isnull().sum()

SaleId                             0
OriginalId                   1133887
PortalId                           0
Title                              0
Description                     8664
Area                            3393
Price                          53424
OfferPrice                   1050699
RealPriceAfterRenovation     1050699
OriginalPrice                1023184
PricePerSquareMeter            57801
NumberOfRooms                  14847
BuiltYear                     382478
Type                               0
BuildingType                  206452
BuildingCondition              14840
OfferFrom                       5031
Floor                          58979
Floors                        505193
TypeOfMarket                   33527
OwnerType                     545555
DateAddedToDatabase                0
DateAdded                          0
DateLastModification          886552
DateLastRaises                835087
NewestDate                         0
AvailableFrom                1133887
L

In [14]:
df_beznull_price = df.dropna(subset=['Area'])

In [15]:
df_beznull_price.isnull().sum()

SaleId                             0
OriginalId                   1130494
PortalId                           0
Title                              0
Description                     7348
Area                               0
Price                          53291
OfferPrice                   1047306
RealPriceAfterRenovation     1047306
OriginalPrice                1019890
PricePerSquareMeter            54408
NumberOfRooms                  12124
BuiltYear                     379203
Type                               0
BuildingType                  204053
BuildingCondition              14800
OfferFrom                       4742
Floor                          56314
Floors                        501959
TypeOfMarket                   32011
OwnerType                     542422
DateAddedToDatabase                0
DateAdded                          0
DateLastModification          883586
DateLastRaises                832330
NewestDate                         0
AvailableFrom                1130494
L

In [16]:
df_beznull_price

,SaleId,OriginalId,PortalId,Title,Description,Area,Price,OfferPrice,RealPriceAfterRenovation,OriginalPrice,...,Archive,Location,VoivodeshipNumber,CountyNumber,CommunityNumber,KindNumber,RegionNumber,SubRegionNumber,StreetNumber,EncryptedId
0,88,NaN,14,Mieszkanie trzypokojowe na sprzedaż,Mieszkanie o powierzchni 73m2 znajduje się na ...,73.00,766500.0,NaN,NaN,NaN,...,NaN,"Podlaskie, Białystok, Bojary",20.0,61.0,1.0,1.0,922490.0,NaN,NaN,02TgImo1wm-Ra4zUQxoPBw==
1,99,NaN,19,Sprzedam mieszkanie na parterze 64.8m2 Białyst...,Sprzedam mieszkanie na parterze 64.8m2 w 3-pię...,64.80,540000.0,NaN,NaN,NaN,...,NaN,"Podlaskie, Białystok, Antoniuk",20.0,61.0,1.0,1.0,922426.0,NaN,NaN,D-NUdVbtN3_kIW7ExgXkvA==
2,115,NaN,3,"Mieszkanie bezczynszowe, 3 pokoje, 2 łazienki",SPRZEDAŻ WYŁĄCZNIE BEZPOŚREDNIA. Agencjom nier...,51.00,540000.0,NaN,NaN,NaN,...,NaN,"Podlaskie, Białystok, Bagnówka",20.0,61.0,1.0,1.0,922461.0,NaN,NaN,lmBSU1XiXLSLOq1yKSe2kA==
3,140,NaN,14,Mieszkanie trzypokojowe na sprzedaż,***Oferta bez prowizji biura i podatku PCC!***...,67.62,544000.0,NaN,NaN,NaN,...,NaN,"Podlaskie, Białystok, Ul. Józefa Karola Puchal...",20.0,61.0,1.0,1.0,922410.0,NaN,43141.0,HWachf5gb6Vq-sFYH4YWnA==
4,145,NaN,3,3 Pokoje- 48M2-Osiedle Dziesięciny,Przedstawiamy na sprzedaż 3 pokojowe mieszkani...,48.00,459000.0,NaN,NaN,NaN,...,NaN,"Podlaskie, Białystok, Ul. Dziesięciny",20.0,61.0,1.0,1.0,922410.0,NaN,4656.0,AWsmt4ZY0X7lNv-6fSCn9Q==
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1133882,4418401,NaN,2,Mieszkanie na sprzedaż - 59 m² - 3 pokoje Grod...,Poznaj naszą inwestycję NA WSPÓLNEJ zlokalizow...,59.99,NaN,NaN,NaN,NaN,...,NaN,"Mazowieckie, Grodziski, Grodzisk Mazowiecki, W...",14.0,5.0,4.0,5.0,2594.0,NaN,24885.0,Ogpi4Jwj8KWDyT0rM_vFYftiMRU9ErLfA6XOKdBw9jY=
1133883,4418402,NaN,2,Mieszkanie na sprzedaż - 41 m² - 2 pokoje Grod...,Poznaj naszą inwestycję NA WSPÓLNEJ zlokalizow...,41.23,NaN,NaN,NaN,NaN,...,NaN,"Mazowieckie, Grodziski, Grodzisk Mazowiecki, W...",14.0,5.0,4.0,5.0,2594.0,NaN,24885.0,Ogpi4Jwj8KWDyT0rM_vFYa7D8z7yjun2jGP_aZIReN8=
1133884,4418404,NaN,2,Mieszkanie na sprzedaż - 49 m² - 3 pokoje Grod...,Poznaj naszą inwestycję NA WSPÓLNEJ zlokalizow...,49.09,NaN,NaN,NaN,NaN,...,NaN,"Mazowieckie, Grodziski, Grodzisk Mazowiecki, W...",14.0,5.0,4.0,5.0,2594.0,NaN,24885.0,Ogpi4Jwj8KWDyT0rM_vFYf9eM73_U8S05NBjfLiez6E=
1133885,4418405,NaN,2,Mieszkanie na sprzedaż - 49 m² - 3 pokoje Grod...,Poznaj naszą inwestycję NA WSPÓLNEJ zlokalizow...,49.48,NaN,NaN,NaN,NaN,...,NaN,"Mazowieckie, Grodziski, Grodzisk Mazowiecki, W...",14.0,5.0,4.0,5.0,2594.0,NaN,24885.0,Ogpi4Jwj8KWDyT0rM_vFYRXVZdKE7iJxxFPu0Bq0ikc=


In [17]:
Q1 = df_beznull_price["Price"].quantile(0.25)
Q3 = df_beznull_price["Price"].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

df_price_p = df_beznull_price[~((df_beznull_price["Price"] < lower_bound) | (df_beznull_price["Price"] > upper_bound))]

In [18]:
Q1 = df_price_p["PricePerSquareMeter"].quantile(0.25)
Q3 = df_price_p["PricePerSquareMeter"].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

df_price_a = df_price_p[~((df_price_p["PricePerSquareMeter"] < lower_bound) | (df_price_p["PricePerSquareMeter"] > upper_bound))]


In [19]:
Q1 = df_price_a["Area"].quantile(0.25)
Q3 = df_price_a["Area"].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

df_price1 = df_price_a[~((df_price_a["Area"] < lower_bound) | (df_price_a["Area"] > upper_bound))]

In [20]:
df_beznull_price.describe().T

,count,mean,std,min,25%,50%,75%,max
SaleId,1130494.0,2.782844e+06,1.215651e+06,88.00,2025160.00,3023760.500,3.763284e+06,4.418406e+06
OriginalId,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PortalId,1130494.0,1.049930e+01,8.739770e+00,1.00,3.00,10.000,1.700000e+01,4.400000e+01
Area,1130494.0,1.377972e+02,4.647859e+04,0.96,40.43,51.000,6.400000e+01,4.878718e+07
Price,1077203.0,2.941016e+06,1.170287e+09,0.00,369000.00,530000.000,7.490000e+05,8.100009e+11
OfferPrice,83188.0,6.281364e+05,3.637220e+05,54000.00,405000.00,555000.000,7.475000e+05,9.600000e+06
RealPriceAfterRenovation,83188.0,1.021383e+06,6.100163e+06,85000.00,518000.00,700873.335,9.750666e+05,3.342342e+08
OriginalPrice,110604.0,1.242231e+10,4.126930e+12,1.00,368000.00,525000.000,7.316310e+05,1.372500e+15
PricePerSquareMeter,1076086.0,1.768959e+04,3.086769e+06,0.00,7566.00,10441.180,1.430000e+04,2.500000e+09
NumberOfRooms,1118370.0,7.805446e+03,2.232357e+06,1.00,2.00,2.000,3.000000e+00,7.925012e+08


In [21]:
df_price1.describe().T

,count,mean,std,min,25%,50%,75%,max
SaleId,1019895.0,2.785335e+06,1.216313e+06,88.00,2.032076e+06,3026910.000,3.765616e+06,4.418406e+06
OriginalId,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PortalId,1019895.0,1.046841e+01,8.734634e+00,1.00,3.000000e+00,10.000,1.700000e+01,4.400000e+01
Area,1019895.0,5.123575e+01,1.515521e+01,6.87,4.000000e+01,49.860,6.100000e+01,9.617000e+01
Price,968696.0,5.347438e+05,2.416853e+05,0.00,3.550000e+05,499000.000,6.806811e+05,1.319000e+06
OfferPrice,76990.0,5.696340e+05,2.311453e+05,54000.00,3.974340e+05,535000.000,6.990000e+05,3.300000e+06
RealPriceAfterRenovation,76990.0,9.404073e+05,6.197016e+06,85000.00,5.036352e+05,676333.330,9.043333e+05,3.342342e+08
OriginalPrice,101360.0,1.423989e+07,3.163320e+09,1.00,3.550000e+05,499000.000,6.840000e+05,8.574609e+11
PricePerSquareMeter,967666.0,1.072436e+04,4.338932e+03,0.00,7.500000e+03,10170.695,1.357357e+04,2.303071e+04
NumberOfRooms,1009140.0,7.482849e+03,2.196238e+06,1.00,2.000000e+00,2.000,3.000000e+00,7.925012e+08


In [22]:
df_price1["Price"].value_counts(normalize=True)

Price
599000.0    0.009037
499000.0    0.008885
399000.0    0.008473
699000.0    0.007991
299000.0    0.006834
              ...   
629788.0    0.000001
626801.0    0.000001
848928.0    0.000001
849517.0    0.000001
374029.0    0.000001
Name: proportion, Length: 117663, dtype: float64

In [23]:
df_price1.sample(10)

,SaleId,OriginalId,PortalId,Title,Description,Area,Price,OfferPrice,RealPriceAfterRenovation,OriginalPrice,...,Archive,Location,VoivodeshipNumber,CountyNumber,CommunityNumber,KindNumber,RegionNumber,SubRegionNumber,StreetNumber,EncryptedId
862699,3796094,NaN,17,"Mieszkanie, ul. Lubelska",KOLEJNY ETAP Mieszkania w stanie deweloperskim...,41.51,409600.0,NaN,NaN,NaN,...,NaN,"Podkarpackie, Rzeszów, Ul. Lubelska",18.0,63.0,1.0,1.0,974133.0,NaN,11307.0,Sf-ho4SvlB_rMOA6UBFPvGDSFhxJitH2UJuyXKHVWIY=
642426,3206439,NaN,4,3-pokojowe mieszkanie na sprzedaż,"Sprzedam mieszkanie o pow. 61m2, zlokalizowane...",61.00,520000.0,NaN,NaN,NaN,...,NaN,"Śląskie, Bielsko-biała, Ul. Jutrzenki",24.0,61.0,1.0,1.0,923584.0,NaN,7699.0,EEOm3YxD32De2oj_SfvpblsXq5Wq09EqXqwZKl7vxi8=
658527,3246392,NaN,18,"Mieszkanie na sprzedaż, 47,2 m2, Piła",Piła - Górne Na sprzedaż mieszkanie 3 pokojowe...,47.20,375000.0,NaN,NaN,NaN,...,NaN,"Wielkopolskie, Pilski, Piła, Górne",30.0,19.0,1.0,1.0,966582.0,NaN,NaN,IvKsAWZRsyqu47ohFpmfg-0vsAuLTfw8YUHtsCxqsbc=
618986,3147290,NaN,3,Nowe mieszkanie w dzielnicy Parkitka,Biuro Sverum Nieruchomości prezentuje nowe mie...,65.16,547344.0,NaN,NaN,NaN,...,NaN,"Śląskie, Częstochowa",24.0,64.0,NaN,NaN,NaN,NaN,NaN,nfho5jwSdz1WLS3SYybzYwdpEJHyF4DkU7ai_0JbJYI=
850010,3763022,NaN,19,Pabianice 3 pokoje ul.Spółdzielcza 45m2 + Gara...,Okazja Cenowa Do sprzedania 3 pokojowe mieszka...,45.00,220000.0,NaN,NaN,NaN,...,NaN,"Łódzkie, Pabianicki, Pabianice",10.0,8.0,2.0,1.0,959079.0,NaN,NaN,sQTKl2ZkeasTR6houRMYr-iSS5I-hJJyKu-R1kSO-hE=
547503,2979736,NaN,11,"Mieszkanie, Warszawa, Bielany, 49 m²","Mieszkanie, Warszawa, Bielany, 49 m² Ogłoszeni...",48.60,827000.0,NaN,NaN,NaN,...,NaN,"Mazowieckie, Warszawa, Bielany",14.0,65.0,4.0,8.0,NaN,NaN,NaN,ixDlMjpECryrZVJzr0Vf6BeiuQCvyTnw2JNa83i7C6w=
547276,2979306,NaN,19,Mieszkanie 2-pokoje Koniuchy,"Na sprzedaż jasne, wygodne 2-pokojowe mieszkan...",39.34,440000.0,NaN,NaN,NaN,...,NaN,"Kujawsko-pomorskie, Toruń",4.0,63.0,NaN,NaN,NaN,NaN,NaN,ztyHMBqMiLs1zGrYYlNERVxslNBbpBprsR-39HjnMC8=
914132,3930901,NaN,11,"Mieszkanie, Bydgoszcz, Górzyskowo, 73 m²","Mieszkanie, Bydgoszcz, Górzyskowo, 73 m² Ogłos...",73.46,734000.0,NaN,NaN,NaN,...,NaN,"Kujawsko-pomorskie, Bydgoszcz, Ul. Górzyskowo",4.0,61.0,1.0,1.0,928363.0,NaN,31411.0,pLVj79hBhne80Wn0YW3SxPtiMRU9ErLfA6XOKdBw9jY=
984729,4116914,NaN,3,Kraków - Krowodrza - ul. Smoluchowskiego,"Mieszkanie 38,11 m²w Krakowie, Krowodrza – ul....",38.11,620000.0,NaN,NaN,NaN,...,NaN,"Małopolskie, Kraków, Kraków-krowodrza, Łobzów",12.0,61.0,2.0,9.0,950581.0,NaN,NaN,1Q6knnbmLjjKkPTBoaeozTY_qkdVfWyqsbDtbsFdaZc=
356105,2366869,NaN,19,Słoneczne mieszkanie trzypokojowe w centrum Wa...,"Świetna lokalizacja, ścisłe centrum Wałcza. Os...",48.40,285000.0,NaN,NaN,NaN,...,NaN,"Zachodniopomorskie, Wałecki, Wałcz",32.0,17.0,1.0,1.0,967245.0,NaN,NaN,_qgff55MgulFf1PWW_HuNzD3XjSU8KMxGYrRgZJdWOU=


In [24]:
df_price1.nunique()

SaleId                       1019895
OriginalId                         0
PortalId                          35
Title                         573126
Description                   882354
Area                            8171
Price                         117663
OfferPrice                     21409
RealPriceAfterRenovation       26948
OriginalPrice                  15696
PricePerSquareMeter           268912
NumberOfRooms                     72
BuiltYear                        265
Type                               2
BuildingType                       4
BuildingCondition                  4
OfferFrom                          2
Floor                             49
Floors                            64
TypeOfMarket                       2
OwnerType                         90
DateAddedToDatabase           976917
DateAdded                     964655
DateLastModification          218341
DateLastRaises                261071
NewestDate                    967284
AvailableFrom                      0
L

In [25]:
unique_btype=df_price1['BuildingType'].unique()

unique_btype

array(['Blok', nan, 'Apartamentowiec', 'Kamienica', 'Pozostałe'],
      dtype=object)

In [26]:
print(df_price1['Price'].nunique())

117663


In [27]:
print(df_price1['Price'].value_counts())

Price
599000.0    8754
499000.0    8607
399000.0    8208
699000.0    7741
299000.0    6620
            ... 
629788.0       1
626801.0       1
848928.0       1
849517.0       1
374029.0       1
Name: count, Length: 117663, dtype: int64


In [28]:
df_price1.isnull().sum()

SaleId                             0
OriginalId                   1019895
PortalId                           0
Title                              0
Description                     6232
Area                               0
Price                          51199
OfferPrice                    942905
RealPriceAfterRenovation      942905
OriginalPrice                 918535
PricePerSquareMeter            52229
NumberOfRooms                  10755
BuiltYear                     355390
Type                               0
BuildingType                  185268
BuildingCondition              13062
OfferFrom                       4574
Floor                          49323
Floors                        456865
TypeOfMarket                   29083
OwnerType                     489606
DateAddedToDatabase                0
DateAdded                          0
DateLastModification          797740
DateLastRaises                754002
NewestDate                         0
AvailableFrom                1019895
L

In [29]:
df_price2 = df_price1.dropna(subset=['Price'])

In [30]:
df_price2.isnull().sum()

SaleId                            0
OriginalId                   968696
PortalId                          0
Title                             0
Description                    6033
Area                              0
Price                             0
OfferPrice                   891706
RealPriceAfterRenovation     891706
OriginalPrice                867336
PricePerSquareMeter            1087
NumberOfRooms                 10648
BuiltYear                    339043
Type                              0
BuildingType                 153500
BuildingCondition             13013
OfferFrom                       789
Floor                         48234
Floors                       436534
TypeOfMarket                  27802
OwnerType                    466260
DateAddedToDatabase               0
DateAdded                         0
DateLastModification         747906
DateLastRaises               717920
NewestDate                        0
AvailableFrom                968696
Link                        

In [31]:
correlation_matrix = df_price2[['Area', 'Price', 'BuiltYear', 'Floor', 'Floors', 'CommunityScore']].corr()

plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Matrix of Price')

Text(0.5, 1.0, 'Correlation Matrix of Price')

In [32]:
sale_ids = df_price2['SaleId'].reset_index(drop=True)

In [34]:
exp = setup(
    data=df_price2,
    target='Price',
    session_id=1122,
    verbose=False,
    categorical_features=[
        'BuildingType',
        'BuildingCondition',
         ],
    keep_features=[
        'SaleId',
        'Location',
        'Area',
         ],
    ignore_features=[
        'RealPriceAfterRenovation',
        'OfferPrice',
        'OriginalPrice',
        'PricePerSquareMeter',
        'OriginalId',
        'PortalId',
        'Title',
        'Type',
        'OfferFrom',
        'TypeOfMarket',
        'OwnerType',
        'DateAddedToDatabase',
        'DateAdded',
        'DateLastModification',
        'DateLastRaises',
        'NewestDate',
        'AvailableFrom',
        'Link',
        'Phone',
        'MainImage',
        'OtherImages',
        'NumberOfDuplicates',
        'NumberOfRaises',
        'NumberOfModifications',
        'IsDuplicatePriceLower',
        'IsDuplicatePrivateOwner',
        'Score',
        'ScorePrecision',
        'CommunityScore',
        'NumberOfCommunityComments',
        'NumberOfCommunityOpinions',
        'Archive',
        'VoivodeshipNumber',
        'CountyNumber',
        'CommunityNumber',
        'RegionNumber',
        'KindNumber',
        'SubRegionNumber',
        'StreetNumber',
        'EncryptedId',
        ],
    ordinal_features={'BuildingType': ['Pozostałe', 'Blok', 'Apartametowiec', 'Kamienica'],
                      'BuildingCondition': ['For_Renovation', 'Good', 'After_Renovation', 'Developer_State' ]},
        
       
)
exp.dataset_transformed.sample(10)

,SaleId,Description,Area,NumberOfRooms,BuiltYear,BuildingType_3.0,BuildingType_-1.0,BuildingType_1.0,BuildingType_0.0,BuildingCondition_-1.0,Floor,Floors,Location,Price
864302,3800382.0,KATOWICE - ZAWODZIE. UL. BOHATERÓW MONTE CASSI...,22.000000,1.0,1978.000000,0.0,0.0,1.0,0.0,1.0,10.000,4.754465,"Śląskie, Katowice, Zawodzie",219000.000
926960,3964816.0,Apartamenty inwestycyjne w Stegnie Zapraszam d...,27.209999,1.0,2025.000000,0.0,1.0,0.0,0.0,1.0,3.000,3.000000,"Pomorskie, Nowodworski, Stegna, Ul. Cicha",568961.125
863156,3797389.0,Blok Na sprzedaż mieszkanie 2-pokojowe o powie...,47.000000,2.0,1970.000000,0.0,0.0,1.0,0.0,1.0,6.000,10.000000,"Świętokrzyskie, Ostrowiecki, Ostrowiec Świętok...",220000.000
997512,4150242.0,Na sprzedaż przestronne mieszkania w prestiżow...,52.450001,3.0,8823.090654,0.0,0.0,1.0,0.0,1.0,6.000,10.000000,"Dolnośląskie, Wrocław, Wrocław-krzyki, Al. Kar...",693073.000
1076259,4316106.0,Na sprzedaż mieszkanie 2-pokojowe o powierzchn...,30.459999,2.0,2012.000000,0.0,0.0,1.0,0.0,1.0,1.000,1.000000,"Pomorskie, Wejherowski, Reda, Ul. Ogrodowa",369000.000
858666,3786195.0,Sprzedam 3-pokojowe mieszkanie o powierzchni 5...,56.000000,3.0,8823.090654,0.0,0.0,1.0,0.0,1.0,0.000,4.754465,"Zachodniopomorskie, Szczecin, Golęcino",430000.000
685673,3313794.0,Mieszkanie w nowym apartamentowcu w samym cent...,39.599998,2.0,2025.000000,0.0,1.0,0.0,0.0,1.0,3.000,4.000000,"Łódzkie, Łódź, Łódź-śródmieście, Śródmieście",431640.000
206458,1517920.0,"""Oferta Prywatna BEZ PROWIZJI dla Kupującego N...",49.759998,3.0,8823.090654,0.0,0.0,1.0,0.0,1.0,2.397,4.754465,"Małopolskie, Kraków, Kraków-krowodrza, Krowodrza",778000.000
286079,2038945.0,Sprzedam mieszkanie w Gołdapi o powierzchni łą...,36.799999,1.0,8823.090654,0.0,0.0,1.0,0.0,1.0,4.000,4.754465,"Warmińsko-mazurskie, Gołdapski, Gołdap",170000.000
302334,2111828.0,APARTAMENTY NA ŁĄKACH USTROŃ Oddanie do użytko...,43.020000,2.0,2024.000000,0.0,1.0,0.0,0.0,1.0,2.000,4.754465,"Śląskie, Cieszyński, Ustroń",535000.000


In [ ]:
unbalanced_exp = setup(
    data=df_price2,
    target='Price',
    verbose=False,
    session_id=1122,
    categorical_features=[
        'BuildingType',
        'BuildingCondition',
         ],
    keep_features=[
        'SaleId',
        'Location',
        'Area',
         ],
    ignore_features=[
        'RealPriceAfterRenovation',
        'OfferPrice',
        'OriginalPrice',
        'PricePerSquareMeter',
        'OriginalId',
        'PortalId',
        'Title',
        'Type',
        'OfferFrom',
        'TypeOfMarket',
        'OwnerType',
        'DateAddedToDatabase',
        'DateAdded',
        'DateLastModification',
        'DateLastRaises',
        'NewestDate',
        'AvailableFrom',
        'Link',
        'Phone',
        'MainImage',
        'OtherImages',
        'NumberOfDuplicates',
        'NumberOfRaises',
        'NumberOfModifications',
        'IsDuplicatePriceLower',
        'IsDuplicatePrivateOwner',
        'Score',
        'ScorePrecision',
        'CommunityScore',
        'NumberOfCommunityComments',
        'NumberOfCommunityOpinions',
        'Archive',
        'VoivodeshipNumber',
        'CountyNumber',
        'CommunityNumber',
        'RegionNumber',
        'KindNumber',
        'SubRegionNumber',
        'StreetNumber',
        'EncryptedId',
        ],
    ordinal_features={'BuildingType': ['Pozostałe', 'Blok', 'Apartametowiec', 'Kamienica'],
                      'BuildingCondition': ['For_Renovation', 'Good', 'After_Renovation', 'Developer_State' ]},
)
best_unbalanced_model = unbalanced_exp.compare_models()
unbalanced_metrics_df = pull()

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,12:13:58
Status,. . . . . . . . . . . . . . . . . .,Fitting 10 Folds
Estimator,. . . . . . . . . . . . . . . . . .,Light Gradient Boosting Machine


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)


Processing:   0%|          | 0/77 [00:00<?, ?it/s]

--- Logging error ---
Traceback (most recent call last):
  File "C:\Users\Ai\AppData\Local\miniconda3\envs\projekt1\Lib\site-packages\pycaret\internal\pycaret_experiment\supervised_experiment.py", line 794, in compare_models
    model, model_fit_time = self._create_model(**create_model_args)
                            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Ai\AppData\Local\miniconda3\envs\projekt1\Lib\site-packages\pycaret\internal\pycaret_experiment\supervised_experiment.py", line 1533, in _create_model
    model, model_fit_time, model_results, _ = self._create_model_with_cv(
                                              ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Ai\AppData\Local\miniconda3\envs\projekt1\Lib\site-packages\pycaret\internal\pycaret_experiment\supervised_experiment.py", line 1126, in _create_model_with_cv
    scores = cross_validate(
             ^^^^^^^^^^^^^^^
  File "C:\Users\Ai\AppData\Local\miniconda3\envs\projekt1\Lib\site-packages\sklearn\util

In [ ]:
plt.figure(figsize=(12, 6))
plt.barh(unbalanced_metrics_df['Model'], unbalanced_metrics_df['R2'], color='skyblue')
plt.xlabel('R2')
plt.title('Porównanie dokładności modeli')
plt.gca().invert_yaxis()  # najlepszy model na górze
plt.grid(True, linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()

In [ ]:
exp.plot_model(best_unbalanced_model, plot='error')

In [ ]:
exp.plot_model(best_unbalanced_model, plot='feature')

In [ ]:
tuned_model = exp.tune_model(best_unbalanced_model, n_iter=3, optimize='R2')

In [ ]:
best_model = exp.compare_models([best_unbalanced_model, tuned_model])

In [ ]:
exp.predict_model(best_unbalanced_model)

In [ ]:
exp.plot_model(best_model, plot='error')

In [ ]:
exp.plot_model(best_unbalanced_model, plot='feature')

In [ ]:
best_tuned_model = exp.tune_model(
    best_unbalanced_model,
    optimize="R2",
    choose_better=True,
    fold=5,
)

In [ ]:
exp.predict_model(best_tuned_model).head(10)

In [ ]:
exp.plot_model(best_tuned_model, plot='error')

In [ ]:
exp.plot_model(best_tuned_model, plot='feature')

In [ ]:
best_final_model = exp.finalize_model(best_unbalanced_model)
best_final_model

In [ ]:
exp.save_model(best_final_model, "0_best_price_modelLGBM", verbose=False);

In [ ]:
exp.plot_model(best_final_model, plot='error')

In [ ]:
exp.predict_model(best_final_model).head(10)

In [ ]:
predictions = exp.predict_model(best_final_model)

In [ ]:
predicted_column = predictions['prediction_label']
print(predicted_column)

In [ ]:
print(predictions[['prediction_label']].applymap('{:,.0f}'.format))

In [ ]:
df_last=predictions[['prediction_label','Price', ]].applymap('{:,.0f}'.format)


In [ ]:
df_last.to_csv('0_new_prices.csv')

In [ ]:
print(df_last.columns)

In [ ]:
print(df_last.head())

In [ ]:
df_last = df_last.reset_index()  
print(df_last.columns)  

In [ ]:
df_last.rename(columns={'index': 'SaleId'}, inplace=True)

In [ ]:
saleflats_df = pd.read_csv('uzupelnione_mieszkania.csv')

In [ ]:
new_prices_df = pd.read_csv('0_new_prices.csv')

In [ ]:
print("saleflats_df.columns:", saleflats_df.columns.tolist())

In [ ]:
print("new_prices_df.columns:", new_prices_df.columns.tolist())

In [ ]:
new_prices_df = new_prices_df.rename(columns={'Unnamed: 0': 'SaleID', 'Price': 'NewPrice'})

In [ ]:
print("new_prices_df.columns:", new_prices_df.columns.tolist())

In [ ]:
print(new_prices_df.columns)

In [ ]:
print(saleflats_df.columns)

In [ ]:
merged_df = pd.merge(
    saleflats_df,
    new_prices_df[['SaleID', 'NewPrice']],
    left_on='SaleId',  # Kolumna w saleflats_df
    right_on='SaleID',  # Kolumna w new_prices_df
    how='left'
)

In [ ]:
merged_df

In [ ]:
# Krok 2: usuwanie dodatkowej kolumny klucza (opcjonalnie, bo mamy już 'SaleId')
merged_df.drop(columns=['SaleID'], inplace=True)

# Krok 3: przestawienie kolumny 'NewPrice' obok 'Price'
cols = merged_df.columns.tolist()

# znajdź indeks kolumny 'Price'
price_index = cols.index('Price')

# usuń NewPrice z listy i dodaj ją zaraz po Price
cols.remove('NewPrice')
cols.insert(price_index + 1, 'NewPrice')

# ustaw nową kolejność kolumn
merged_df = merged_df[cols]

In [ ]:
merged_df

In [ ]:
merged_df[merged_df.duplicated()]

In [ ]:
prediction_df = merged_df.copy()

In [ ]:
from pycaret.regression import predict_model

In [ ]:
prediction_df_clean = prediction_df.drop(columns=['Price'], errors='ignore')
predictions = predict_model(best_final_model, data=prediction_df_clean)

In [ ]:
predictions['RealPrice'] = prediction_df['Price']

In [ ]:
merged_df['PredictedPrice'] = predictions['prediction_label']

In [ ]:
predictions=predictions[['prediction_label',]].applymap('{:,.0f}'.format)

In [ ]:
# Przenieś kolumnę 'PredictedPrice' za 'NewPrice'
cols = list(merged_df.columns)
new_price_index = cols.index('NewPrice')
# Usuń z listy kolumn
cols.remove('PredictedPrice')
# Dodaj w odpowiednie miejsce
cols.insert(new_price_index + 1, 'PredictedPrice')
# Przekształć DataFrame
merged_df = merged_df[cols]

In [ ]:
merged_df.to_csv('0_new_prices_full.csv')

In [ ]:
merged_df = pd.read_csv('0_new_prices_full.csv')

In [ ]:
merged_df

In [ ]:
merged_df2=merged_df[['PredictedPrice',]].applymap('{:,.0f}'.format)

In [ ]:
merged_df2

In [ ]:
merged_df['PredictedPrice'] = merged_df2['PredictedPrice']

In [ ]:
merged_df.drop(columns=['NewPrice'], inplace=True)

In [ ]:
merged_df

In [ ]:
merged_df.to_csv('uzupelnione_mieszkania_ceny.csv')

In [ ]:
# df_p = pd.read_csv('sale_2024_6.csv')

# 1. Kopia oryginalnej dużej tabeli
df = df_p.copy()

# 2. Dopinamy kolumnę z predykcjami po SaleId
if 'NewPrices' in merged_df.columns:
    df = df.merge(merged_df[['SaleId', 'NewPrices']], on='SaleId', how='left')
elif 'PredictedPrice' in merged_df.columns:
    df = df.merge(merged_df[['SaleId', 'PredictedPrice']].rename(columns={'PredictedPrice': 'NewPrices'}), on='SaleId', how='left')
else:
    raise ValueError("W merged_df nie znaleziono kolumny 'NewPrices' ani 'PredictedPrice'")

# 3. Tam, gdzie jest dostępna cena ('Price'), kopiujemy ją do 'NewPrices'
df.loc[df['Price'].notna(), 'NewPrices'] = df.loc[df['Price'].notna(), 'Price']

# 4. Przenosimy kolumnę 'NewPrices' za 'Price'
cols = df.columns.tolist()
if 'NewPrices' in cols:
    cols.remove('NewPrices')
    price_idx = cols.index('Price')
    cols.insert(price_idx + 1, 'NewPrices')
    df = df[cols]

# 5. Załaduj model z PyCaret
model = load_model('6_best_price_modelLGBM')

# 6. Przygotuj dane bez kolumny 'Price', by uniknąć błędu z NaN
df_for_prediction = df.drop(columns=['Price'], errors='ignore')

# 7. Przeprowadź predykcję
predicted_df = predict_model(model, data=df_for_prediction)

# 8. Powiąż wynik predykcji po SaleID
df = df.merge(predicted_df[['SaleID', 'prediction_label']], on='SaleID', how='left', suffixes=('', '_new'))

# 9. Nadpisz kolumnę 'NewPrices' nowymi przewidywaniami
df['NewPrices'] = df['prediction_label']

# 10. Przenosimy kolumnę 'NewPrices' za 'Price' (jeszcze raz)
cols = df.columns.tolist()
if 'NewPrices' in cols:
    cols.remove('NewPrices')
    price_idx = cols.index('Price')
    cols.insert(price_idx + 1, 'NewPrices')
    df = df[cols]

# 11. Zaktualizuj saleflats_b
saleflats_b = df

df2=df[['NewPrices',]].applymap('{:,.0f}'.format)

# Nadpisanie kolumny 'NewPrices' w df sformatowaną wersją
df['NewPrices'] = df2['NewPrices']

df

df.to_csv('6_new_prices_FULL2.csv')

df.sample(100)